In [1]:
import os 
import numpy as np
import pandas as pd
import scipy.io
import seaborn as sns

from neo.core import SpikeTrain
from quantities import ms, s, Hz
from elephant.statistics import mean_firing_rate
from elephant.statistics import time_histogram, instantaneous_rate
from elephant.kernels import GaussianKernel


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:

for monkey_directory in ['Speriatus', 'Mojo']:
    path = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Gottlib_data\\data files\\distractor paper data only\\' +monkey_directory
    ### path = 'C:\\Users\\David\\Desktop\\fast_process_gottlieb\\' +monkey_directory
    Monkey = path.split('\\')[-1]
    for file_to_use in os.listdir(path):  #
        neuron = file_to_use.split('.')[0]
        print(Monkey, neuron)
        #########
        ### Open file
        f = scipy.io.loadmat(os.path.join(path, file_to_use))
        ########
        ### Check if there is spiking data inside
        if np.shape(f['data']['spikes'][0][0][0]) ==(0,): 
            print('No data')
            
        else:
            ########
            ### Spikes 
            df_spike_time=pd.DataFrame(f['data']['spikes'][0][0][0][0][:, :])
            df_spike_time.columns = [str(i) for i in range(0, np.shape(df_spike_time)[1])]
            ########
            ### Events
            Dict_events={}
            for i in range(0, len(f['data']['events'][0][0][0])     ):
                events = pd.DataFrame( f['data']['events'][0][0][0][i])
                events.columns=['time', 'code']
                Dict_events[str(i)]= events
            #
            ########
            ### Descriptors & Bad trials
            ## Use the Descriptors to make the booleans to select the HIT trials
            lists=[]
            for T in range(0, len(f['data']['descriptors'][0][0][0])):
                lists.append( [f['data']['descriptors'][0][0][0][T][i][0] for i in range(len(f['data']['descriptors'][0][0][0][T]) )] )
            #
            Descriptors = pd.DataFrame(lists)
            Descriptors=Descriptors.transpose()
            Descriptors.columns = [str(i) for i in range(0, np.shape(Descriptors)[1])]
            # 6 "failed" // performance code, as defined below 
            boolean_hit_trials = Descriptors.iloc[6].isin([1]) #Descriptors.iloc[6,:]==1
            ########
            ### Bad trials
            ## take off the bad trials
            bad = pd.DataFrame(np.array([f['data']['bad'][0][0][0][x][0][0] for x in range(0, len(f['data']['bad'][0][0][0]))]))
            bad = bad.transpose()
            bad.columns = [str(i) for i in range(0, np.shape(df_spike_time)[1])]
            #each column in a trial; if 1, discard it
            boolean_bad_trials = bad.iloc[0,:]!=1
            ########
            ########
            ### Boolean combinging success trials and correct
            boolean_keep = boolean_hit_trials & boolean_bad_trials 
            print('Trials keept = ' +str(sum(boolean_keep)))
            print('Trials excluded = ' + str(np.shape(df_spike_time)[1] - sum(boolean_keep) ))
            ########
            ### df_spikes_correct
            indexes_trials_keep = np.array([str(i) for i in range(0, np.shape(df_spike_time)[1])])[boolean_keep]
            df_spikes = df_spike_time[list(indexes_trials_keep)]
            df_spikes = df_spikes.iloc[4:, :] #remove the first 4 rows, no spikes
            df_spikes = df_spikes.transpose()[df_spikes.iloc[0,:]<9999].transpose() #remove columns full of nans
            ########
            ########
            ########
            ### Get the firing rate of each trial (Gaussian fit and auto)
            ### turorial https://elephant.readthedocs.io/en/latest/tutorials/statistics.html
            trials_=[]            
            Descriptors_use = Descriptors[list(df_spikes.columns)]           
            for TRIAL in list(df_spikes.columns):
                ###### firing rates
                times_spikes_all = df_spikes[TRIAL].values # get trial spike times
                times_spikes = times_spikes_all[times_spikes_all<9999] ##remove nans in the trial (at the end)
                times_spikes = times_spikes[times_spikes>0] #no negative timings for spikes
                stop_time =  times_spikes.max()
                spiketrain1 = SpikeTrain(times_spikes, units=ms, t_stop=stop_time) #spiketrain
                histogram_rate = time_histogram([spiketrain1], 10*ms, output='rate') #sampling rate of 10 Suzuki & Gottlieb, increase for less temporal resolution. 
                #inst_rate = instantaneous_rate(spiketrain1, sampling_period=10*ms) 
                gaus_rate = instantaneous_rate(spiketrain1, sampling_period=10*ms, kernel=GaussianKernel(15*ms)) #s.d of Suzuki & Gottlieb 
                times_ = gaus_rate.times.rescale(ms)
                #firing_auto = inst_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()
                firing_gauss = gaus_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()
                df_trial = pd.DataFrame({'times':times_, 'firing_gauss':firing_gauss}) #, 'firing_auto': firing_auto})
                                
                ####### descriptive data
                df_trial['trial']=TRIAL 
                df_trial['TDOA'] = Descriptors_use[TRIAL].iloc[20]
                df_trial['TDconf'] = Descriptors_use[TRIAL].iloc[24]
                df_trial['targetloc'] = Descriptors_use[TRIAL].iloc[1]
                df_trial['distLoc'] = Descriptors_use[TRIAL].iloc[19]
                df_trial['fixationtime'] = Descriptors_use[TRIAL].iloc[9]
                df_trial['failed'] = Descriptors_use[TRIAL].iloc[6]
                df_trial['saccadeChoice'] = Descriptors_use[TRIAL].iloc[18]
                df_trial['cueDuration'] = Descriptors_use[TRIAL].iloc[10]
                df_trial['distDur'] = Descriptors_use[TRIAL].iloc[21]
                df_trial['monkey'] = Monkey
                df_trial['neuron'] = neuron           
                trials_.append(df_trial)
            #
            df_neuron=pd.concat(trials_)
            ########
            ########
            ### Normalization
            # Get the mean of each time in the control condition (T in RF no dist), then get the max of this means
            max_fr_gauss_normalize = df_neuron.loc[(df_neuron['TDconf']==0), ['firing_gauss', 'times']].groupby('times').describe()['firing_gauss']['mean'].max()
            #max_fr_auto_normalize = df_neuron.loc[(df_neuron['TDconf']==0), ['firing_auto', 'times']].groupby('times').describe()['firing_auto']['mean'].max()
            df_neuron['norm_firing_gauss'] = df_neuron['firing_gauss']/max_fr_gauss_normalize
            #df_neuron['norm_firing_auto'] = df_neuron['firing_auto']/max_fr_auto_normalize
            ########
            ########
            ######## Save the dataframe as an excel with the name of the neuron
            path_save = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Gottlib_data\\firing_rates'
            path_save_neuron = os.path.join(path_save, neuron+'.xlsx')
            df_neuron.to_excel(path_save_neuron)




Speriatus 189139306
Trials keept = 313
Trials excluded = 315


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in less
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 1 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 2 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:990: UserWarning: Correcting 1 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  'behaviour.'.format(num_rounding_corrections))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:274: UserWarning: Correcting a rounding error in the calculation of n_bins by increasing n_bins by 1. You 